# Encode Datasets

This notebook takes as input a trained encoder, target set, and query set, and saves encoded targets & queries, as well the query-target distance matrix.

In [1]:

# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import tensorflow as tf
import pandas as pd

import primo.models
import primo.datasets
import primo.tools.filepath as filepaths

from tqdm.notebook import tqdm

Reserve space on the GPU for running simulations. It's important to do this before running any tensorflow code (which will take all available GPU memory):

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [2]:
encoder_filepath = filepaths.get_encoder_model_path(isDocker=True)
# encoder = primo.models.Encoder('/tf/primo/data/models/encoder-function.h5')
encoder = primo.models.Encoder('/tf/primo/data/models/encoder_1site_triplet_20220131_1.h5')
print(filepaths.get_encoder_model_path(True))

/tf/primo/data/models/encoder-model.h5


### Encode Queries
This code loads the query features, encodes them to DNA sequences, and saves the result.

In [3]:
query_features_filepath = filepaths.get_query_features_path(isDocker=True)
query_features = pd.read_hdf(query_features_filepath)

In [4]:
query_sequence_save_path = filepaths.get_encoded_query_sequences_path(isDocker=True)
query_seqs = encoder.encode_feature_seqs(query_features)
pd.DataFrame(
    query_seqs, index=query_features.index, columns=['FeatureSequence']
).to_hdf(
    query_sequence_save_path, key='df', mode='w'
)

In [5]:
for s in query_seqs:
    print(s)

ATCGCGTGATCAACCACCGC
ATGCCGTGATCAACCATCGC
ATACTGTGAAAAACCATCCC


### Encode Target Set
This code loads the target set's features, calculates distances to each query, encodes them to DNA sequences, and saves the result.

In [7]:
# Memory-mapped file that caches the distances between targets and queries
distance_store_path = filepaths.get_distance_store_path(isDocker=True)
dist_store = pd.HDFStore(distance_store_path, complevel=9, mode='w')

# Memory-mapped file that stores the DNA sequence encodings of the target features.
sequence_store_path = filepaths.get_sequence_store_path(isDocker=True)
seq_store = pd.HDFStore(sequence_store_path, complevel=9, mode='w')

try:
    # Target images are split up across 16 files.
    # Because these files are so large, and can't all be stored into memory on a single machine, 
    # there's some low-level memory-management wizardly happening below.
    prefixes = [ "%x"%i for i in range(16) ]
    for prefix in tqdm(prefixes):
        target_feature_path = filepaths.get_target_feature_path(prefix, isDocker=True)
        target_features = pd.read_hdf(target_feature_path)

        # Dictionary that maps queries to euclidean distances for every pairing of query and target.
        distances = {}
        for query_id, query in query_features.iterrows():
            # Calculuate the Euclidean distance between each query and target.
            distances[query_id] = np.sqrt(np.square(target_features.values - query.values).sum(1))
            

        df = pd.DataFrame(distances, index=target_features.index)
        dist_store.append('df', df)

        # Low-level memory mangement
        del df, distances
        
        target_seqs = encoder.encode_feature_seqs(target_features)
        df = pd.DataFrame(target_seqs, index=target_features.index, columns=['FeatureSequence'])
        seq_store.append('df', df)
        del df, target_seqs
        
        del target_features

finally:
    dist_store.close()
    seq_store.close()

  0%|          | 0/16 [00:00<?, ?it/s]

### Encode Extended Target Set
This code loads the extended target set's features, encodes them to DNA sequences, calculates distances to each query, and saves the result.

In [9]:
# 130027 / 1743042 (7.5%) # Images from the full set of ~9 million images that are not used in either the
# "target" set or the "train" set. These are not hosted by the CVDF.

distance_store_path = filepaths.get_extended_target_distance_store_path(isDocker=True)
dist_store = pd.HDFStore(distance_store_path, complevel=9, mode='w')

sequence_store_path = filepaths.get_extended_sequence_store_path(isDocker=True)
seq_store = pd.HDFStore(sequence_store_path, complevel=9, mode='w')

try:
    # Note: Prefixes are lower-cased hexidecimal (e.g. '4b').
    prefixes = [ "%x"%i for i in range(16) ]
    for prefix in tqdm(prefixes):
        target_features_path = filepaths.get_extended_target_feature_path(prefix, isDocker=True)
        target_features = pd.read_hdf(target_features_path)

        distances = {}
        for query_id, query in query_features.iterrows():
            distances[query_id] = np.sqrt(np.square(target_features.values - query.values).sum(1))

        df = pd.DataFrame(distances, index=target_features.index)
        dist_store.append('df', df)
        del df, distances
        
        target_seqs = encoder.encode_feature_seqs(target_features)
        df = pd.DataFrame(target_seqs, index=target_features.index, columns=['FeatureSequence'])
        seq_store.append('df', df)
        del df, target_seqs
        
        del target_features

finally:
    dist_store.close()
    seq_store.close()

  0%|          | 0/16 [00:00<?, ?it/s]

MemoryError: Unable to allocate 4.77 GiB for an array with shape (312816, 4096) and data type float32